# Data Wrangling

In [2]:
import pandas as pd

# Show all columns
pd.set_option('display.max_columns', None)

In [3]:
data = pd.read_excel('data/data.xlsx')

In [4]:
# Calculate score of each subscale
## PT: personality: 3, 8, 11, 15, 21, 25 and 28
## FS: personality: 1, 5, 7, 12, 16, 23 and 26
## EC: personality: 2, 4, 9, 14, 18, 20 and 22
## PD: personality: 6, 10, 13, 17, 19, 24 and 27
select_PT = [f'personality:{i}' for i in [3, 8, 11, 15, 21, 25, 28]]
select_FS = [f'personality:{i}' for i in [1, 5, 7, 12, 16, 23, 26]]
select_EC = [f'personality:{i}' for i in [2, 4, 9, 14, 18, 20, 22]]
select_PD = [f'personality:{i}' for i in [6, 10, 13, 17, 19, 24, 27]]

data['PT'] = data[select_PT].sum(axis=1)/7
data['FS'] = data[select_FS].sum(axis=1)/7
data['EC'] = data[select_EC].sum(axis=1)/7
data['PD'] = data[select_PD].sum(axis=1)/7

In [5]:
data['country:1'] = data['country:1'].str.lower()

In [6]:
data['country:1'] = data['country:1'].replace('usa', 'united states')
data['country:1'] = data['country:1'].replace('united states of america', 'united states')
data['country:1'] = data['country:1'].replace('italy ', 'italy')

In [7]:
print(data['country:1'].unique().shape)

(17,)


In [8]:
#print all unique values of country
data['country:1'].unique()

array(['israel', 'latvia', 'italy', 'france', 'cyprus ', 'china',
       'switzerland', 'greece', 'ireland', 'australia', 'united kingdom',
       'palestine', 'morroco ', 'cyprus', 'syria', 'united states',
       'hungary'], dtype=object)

In [9]:
data['genre:1'] = data['genre:1'].replace([1, 2, 3], ['M', 'F', 'O'])
for i in range(1, 13):
    data[f'Q{i}:1'] = data[f'Q{i}:1'].replace({1: True, 2: False}).astype('bool')


In [10]:
select_col = [f'Q{i}:1' for i in range(1, 13)]

data['Truth Count'] = data[select_col].sum(axis=1).astype(int)
data['Lies Count'] = (12 - data['Truth Count']).astype(int)

# Split english proficiency into categories -> How many?

In [11]:
# On average how confident are people when answering truth/lie
data['Avg Confidence Truth'] = 0
data['Avg Confidence Lie'] = 0
data['Avg Confidence General'] = 0



# calculate average confidence for True answers
for i in range(1, 13):
    q_col = f'Q{i}:1'
    q_conf_col = f'Q{i}_2:1'
    data.loc[data[q_col], 'Avg Confidence Truth'] += data[q_conf_col] / data['Truth Count']

# calculate average confidence for False answers
for i in range(1, 13):
    q_col = f'Q{i}:1'
    q_conf_col = f'Q{i}_2:1'
    data.loc[~data[q_col], 'Avg Confidence Lie'] += data[q_conf_col] / data['Lies Count']

# calculate average confidence for all answers
for i in range(1, 13):
    q_conf_col = f'Q{i}_2:1'
    data['Avg Confidence General'] += data[q_conf_col] / 12



In [12]:
veracity = [True, True, True, True, True, False, False, False, False, False, False, True]
len(veracity)
data["Accuracy"] = 0
accuracy = 0
for i in range(1, 13):
    q_col = f'Q{i}:1'
    data.loc[data[q_col] == veracity[i-1], 'Accuracy'] += 1/len(veracity)



In [ ]:
leakage = []

In [13]:
data.head(1)


,participant,age:1,genre:1,english:1,country:1,personality:1,personality:2,personality:3,personality:4,personality:5,personality:6,personality:7,personality:8,personality:9,personality:10,personality:11,personality:12,personality:13,personality:14,personality:15,personality:16,personality:17,personality:18,personality:19,personality:20,personality:21,personality:22,personality:23,personality:24,personality:25,personality:26,personality:27,personality:28,Q1:1,Q1_2:1,Q2:1,Q2_2:1,Q3:1,Q3_2:1,Q4:1,Q4_2:1,Q5:1,Q5_2:1,Q6:1,Q6_2:1,Q7:1,Q7_2:1,Q8:1,Q8_2:1,Q9:1,Q9_2:1,Q10:1,Q10_2:1,Q11:1,Q11_2:1,Q12:1,Q12_2:1,TIME_start,TIME_end,TIME_total,PT,FS,EC,PD,Truth Count,Lies Count,Avg Confidence Truth,Avg Confidence Lie,Avg Confidence General,Accuracy
0,s.54cdd6e1-ce13-449a-a7bf-ce45faff34a1.txt,22,F,10,israel,5,4,2,3,5,3,5,4,3,5,3,5,4,3,4,5,3,3,1,4,2,5,5,1,3,4,1,3,False,51,False,60,True,78,False,60,False,59,True,66,True,78,False,50,False,50,True,69,False,80,False,80,2023-04-15-21-26,2023-04-15-21-57,31,3.0,4.857143,3.571429,2.571429,4,8,72.75,61.25,65.083333,0.333333


In [16]:
clean_data = data[['country:1', 'genre:1', 'age:1', 'english:1', 'PT', 'FS', 'EC', 'PD', 'Accuracy', 'Truth Count', 'Lies Count', 'Avg Confidence Truth', 'Avg Confidence Lie', 'Avg Confidence General', 'TIME_total']]

In [17]:
# Rename columns
clean_data = clean_data.rename({'country:1': 'Country', 'genre:1': 'Sex', 'english:1': 'English Proficiency', 'age:1': 'Age', 'TIME_total': 'Time Spent (min)'}, axis=1)

# Exoport to csv
clean_data.to_csv('data/clean_data.csv', index=False)

In [18]:
clean_data.head()

,Country,Sex,Age,English Proficiency,PT,FS,EC,PD,Accuracy,Truth Count,Lies Count,Avg Confidence Truth,Avg Confidence Lie,Avg Confidence General,Time Spent (min)
0,israel,F,22,10,3.000000,4.857143,3.571429,2.571429,0.333333,4,8,72.750000,61.250000,65.083333,31
1,latvia,M,22,10,3.285714,4.000000,3.571429,1.857143,0.416667,9,3,74.555556,84.000000,76.916667,12
2,italy,M,23,9,4.428571,3.142857,3.142857,2.142857,0.583333,9,3,67.777778,70.000000,68.333333,8
3,italy,M,24,7,2.857143,4.000000,3.428571,2.571429,0.500000,6,6,71.833333,61.500000,66.666667,11
4,france,F,22,9,2.285714,4.285714,3.571429,3.000000,0.500000,6,6,62.833333,78.833333,70.833333,9


In [19]:
data.head(50)

,participant,age:1,genre:1,english:1,country:1,personality:1,personality:2,personality:3,personality:4,personality:5,personality:6,personality:7,personality:8,personality:9,personality:10,personality:11,personality:12,personality:13,personality:14,personality:15,personality:16,personality:17,personality:18,personality:19,personality:20,personality:21,personality:22,personality:23,personality:24,personality:25,personality:26,personality:27,personality:28,Q1:1,Q1_2:1,Q2:1,Q2_2:1,Q3:1,Q3_2:1,Q4:1,Q4_2:1,Q5:1,Q5_2:1,Q6:1,Q6_2:1,Q7:1,Q7_2:1,Q8:1,Q8_2:1,Q9:1,Q9_2:1,Q10:1,Q10_2:1,Q11:1,Q11_2:1,Q12:1,Q12_2:1,TIME_start,TIME_end,TIME_total,PT,FS,EC,PD,Truth Count,Lies Count,Avg Confidence Truth,Avg Confidence Lie,Accuracy
0,s.54cdd6e1-ce13-449a-a7bf-ce45faff34a1.txt,22,F,10,israel,5,4,2,3,5,3,5,4,3,5,3,5,4,3,4,5,3,3,1,4,2,5,5,1,3,4,1,3,False,51,False,60,True,78,False,60,False,59,True,66,True,78,False,50,False,50,True,69,False,80,False,80,2023-04-15-21-26,2023-04-15-21-57,31,3.000000,4.857143,3.571429,2.571429,4,8,72.750000,61.250000,0.333333
1,s.ed5d036d-85c5-4019-b45b-0f4c77671f68.txt,22,M,10,latvia,4,4,5,1,2,2,2,3,4,3,4,5,2,4,2,5,3,5,1,4,1,3,5,1,3,5,1,5,True,84,False,74,False,97,True,54,True,75,True,53,True,94,True,97,False,81,True,85,True,62,True,67,2023-04-19-16-59,2023-04-19-17-11,12,3.285714,4.000000,3.571429,1.857143,9,3,74.555556,84.000000,0.416667
2,s.8c483281-2b26-44ac-90eb-0fe6d6494cde.txt,23,M,9,italy,4,4,4,3,3,2,2,4,3,2,5,4,3,3,5,4,2,3,2,4,5,2,3,2,4,2,2,4,True,70,True,80,True,55,False,70,True,55,False,80,False,60,True,65,True,60,True,85,True,90,True,50,2023-04-06-20-52,2023-04-06-21-00,8,4.428571,3.142857,3.142857,2.142857,9,3,67.777778,70.000000,0.583333
3,s.bc60d408-b74e-45f7-a191-881d54d06c4c.txt,24,M,7,italy,3,4,3,3,4,2,4,2,3,2,4,5,2,3,2,4,5,5,3,3,2,3,4,1,3,4,3,4,True,64,False,57,False,61,True,70,True,66,False,61,True,61,True,100,True,70,False,61,False,66,False,63,2023-04-06-20-01,2023-04-06-20-12,11,2.857143,4.000000,3.428571,2.571429,6,6,71.833333,61.500000,0.500000
4,s.16fd83ad-7200-44d1-ae9f-1345d3fdb1b2.txt,22,F,9,france,5,4,1,3,1,2,4,1,3,4,4,5,4,5,2,5,4,3,2,4,2,3,5,2,1,5,3,5,False,75,True,50,False,75,True,50,True,50,False,95,False,78,True,76,True,75,True,76,False,75,False,75,2023-04-06-19-57,2023-04-06-20-06,9,2.285714,4.285714,3.571429,3.000000,6,6,62.833333,78.833333,0.500000
5,s.c8831bcc-0972-4eb7-9171-8f8209aa7b3d.txt,29,F,7,italy,5,5,4,4,5,4,2,5,4,5,4,5,2,4,2,4,3,4,2,4,5,5,5,2,4,5,4,5,True,63,False,53,False,63,True,65,True,66,False,58,False,59,True,63,False,69,False,59,True,57,True,53,2023-04-12-19-36,2023-04-12-19-52,16,4.142857,4.428571,4.285714,3.142857,6,6,61.166667,60.166667,0.666667
6,s.02d5da94-d14a-4c5b-8a47-f5326c9d087f.txt,51,M,9,cyprus,2,4,4,3,2,3,2,4,3,4,4,1,2,3,2,1,3,5,3,3,4,3,1,2,4,2,3,4,False,30,True,35,False,25,False,20,True,49,True,45,False,69,False,45,False,50,True,40,True,51,False,55,2023-04-16-18-22,2023-04-16-18-46,24,3.714286,1.571429,3.428571,2.857143,5,7,44.000000,42.000000,0.416667
7,s.8af8c273-aab1-4940-9ccd-fbf0c566aad2.txt,22,M,8,italy,5,4,4,4,2,3,4,5,4,5,4,4,4,5,1,3,4,4,4,5,5,4,3,4,2,5,5,3,False,44,True,53,True,86,True,60,True,38,True,79,False,86,False,69,False,82,False,50,False,63,True,79,2023-04-12-21-14,2023-04-12-21-23,9,3.428571,3.714286,4.285714,4.142857,6,6,65.833333,65.666667,0.833333
8,s.afc9c4c5-05a2-4f08-884a-ec9ce2dc797a.txt,22,F,8,italy,5,3,1,3,3,3,3,3,5,3,3,3,3,3,3,4,3,3,2,3,3,3,3,3,3,3,3,3,False,24,False,23,True,39,False,26,True,36,False,23,True,36,True,44,True,54,True,31,False,34,True,61,2023-04-06-20-15,2023-04-06-20-23,8,2.714286,3.428571,3.285714,2.857143,7,5,43.000000,26.000000,0.416667
9,s.122d2e1b-f87d-4858-b361-1994241a4fe1.txt,22,F,9,china,3,4,4,4,4,3,3,3,2,3,4,1,3,2,2,2,3,4,3,4,5,3,2,3,3,5,3,4,True,79,True,82,False,72,False,73,False,78,True,52,False,37,True,73,False,71,True,70,True,83,False,70,2023-04-06-21-43,2023-04-06-21-50,7,3.571429,2.857143,3.285714,3.000000,6,6,73.166667,66.833333,0.333333
